In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import io

In [2]:
#Import Labels
directory="G:\\Big Download\\CheXpert-v1.0\\train_visualCheXbert_edit.csv"


In [3]:
train_labels = pd.read_csv(directory)

In [4]:
train_labels

,Path,Header,Type,Patient,Study,View,Sex,Age,Frontal/Lateral,AP/PA,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/train/patient00001/study1/view1_...,CheXpert-v1.0,train,patient00001,study1,view1_frontal.jpg,Female,68,Frontal,AP,...,0,0,0,0,0,0,0,0,1,0
1,CheXpert-v1.0/train/patient00002/study1/view1_...,CheXpert-v1.0,train,patient00002,study1,view1_frontal.jpg,Female,83,Frontal,AP,...,0,1,1,1,0,0,1,1,0,0
2,CheXpert-v1.0/train/patient00002/study1/view2_...,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg,Female,83,Lateral,NaN,...,0,1,1,1,0,0,1,1,0,0
3,CheXpert-v1.0/train/patient00002/study2/view1_...,CheXpert-v1.0,train,patient00002,study2,view1_frontal.jpg,Female,87,Frontal,AP,...,1,1,0,1,0,1,0,0,0,0
4,CheXpert-v1.0/train/patient00003/study1/view1_...,CheXpert-v1.0,train,patient00003,study1,view1_frontal.jpg,Male,41,Frontal,AP,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0/train/patient64537/study1/view1_...,CheXpert-v1.0,train,patient64537,study1,view1_frontal.jpg,Male,59,Frontal,AP,...,0,1,1,1,0,1,1,1,0,0
223410,CheXpert-v1.0/train/patient64537/study2/view1_...,CheXpert-v1.0,train,patient64537,study2,view1_frontal.jpg,Male,59,Frontal,AP,...,0,0,0,1,1,1,0,1,1,0
223411,CheXpert-v1.0/train/patient64538/study1/view1_...,CheXpert-v1.0,train,patient64538,study1,view1_frontal.jpg,Female,0,Frontal,AP,...,1,0,0,0,0,0,0,0,0,0
223412,CheXpert-v1.0/train/patient64539/study1/view1_...,CheXpert-v1.0,train,patient64539,study1,view1_frontal.jpg,Female,0,Frontal,AP,...,1,1,0,1,0,0,0,0,1,0


In [ ]:
#new_cols=train_labels['Path'].str.split('/', expand=True)

In [ ]:
#name_map={0:"header",1: "Type",2: "Patient",3: "Study",4:"View"}
#new_cols_rename=new_cols.rename(name_map,axis=1)

In [ ]:
#new_cols_rename

In [ ]:
#extra_labels=pd.concat([train_labels,new_cols_rename],axis=1)

In [ ]:
#train_labels

In [ ]:
#extra_labels

In [ ]:
#Pseudocode
#Loop through dataframe -> look ahead -> if the next row is lateral, index the two images and then skip. If the next is front, just move forward

In [5]:
#laterals_images=extra_labels.loc[extra_labels["View"]=="view2_lateral.jpg"]
laterals_images=train_labels.loc[train_labels["View"]=="view2_lateral.jpg"]

In [6]:
laterals_images

,Path,Header,Type,Patient,Study,View,Sex,Age,Frontal/Lateral,AP/PA,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
2,CheXpert-v1.0/train/patient00002/study1/view2_...,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg,Female,83,Lateral,NaN,...,0,1,1,1,0,0,1,1,0,0
6,CheXpert-v1.0/train/patient00004/study1/view2_...,CheXpert-v1.0,train,patient00004,study1,view2_lateral.jpg,Female,20,Lateral,NaN,...,0,0,0,0,0,0,0,0,0,1
8,CheXpert-v1.0/train/patient00005/study1/view2_...,CheXpert-v1.0,train,patient00005,study1,view2_lateral.jpg,Male,33,Lateral,NaN,...,0,0,0,0,0,0,0,0,1,0
17,CheXpert-v1.0/train/patient00009/study1/view2_...,CheXpert-v1.0,train,patient00009,study1,view2_lateral.jpg,Male,76,Lateral,NaN,...,0,0,0,0,0,0,0,0,0,0
19,CheXpert-v1.0/train/patient00010/study1/view2_...,CheXpert-v1.0,train,patient00010,study1,view2_lateral.jpg,Female,50,Lateral,NaN,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219187,CheXpert-v1.0/train/patient60703/study1/view2_...,CheXpert-v1.0,train,patient60703,study1,view2_lateral.jpg,Male,88,Lateral,NaN,...,1,1,0,1,0,1,0,0,1,0
219189,CheXpert-v1.0/train/patient60704/study1/view2_...,CheXpert-v1.0,train,patient60704,study1,view2_lateral.jpg,Male,67,Lateral,NaN,...,0,0,0,0,0,0,0,0,0,1
219193,CheXpert-v1.0/train/patient60706/study1/view2_...,CheXpert-v1.0,train,patient60706,study1,view2_lateral.jpg,Female,99,Lateral,NaN,...,1,0,0,0,0,0,0,0,1,0
219195,CheXpert-v1.0/train/patient60707/study1/view2_...,CheXpert-v1.0,train,patient60707,study1,view2_lateral.jpg,Female,38,Lateral,NaN,...,0,0,0,0,0,0,0,0,1,0


In [7]:
lateral_index_list=laterals_images.index.tolist()

In [9]:
front_index=laterals_images.index-1

In [10]:
front_index_list=front_index.tolist()

In [11]:
target_image_index=lateral_index_list+front_index_list

In [12]:
len(target_image_index)

61266

In [13]:
paired_images=train_labels.iloc[target_image_index]

In [14]:
paired_images

,Path,Header,Type,Patient,Study,View,Sex,Age,Frontal/Lateral,AP/PA,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
2,CheXpert-v1.0/train/patient00002/study1/view2_...,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg,Female,83,Lateral,NaN,...,0,1,1,1,0,0,1,1,0,0
6,CheXpert-v1.0/train/patient00004/study1/view2_...,CheXpert-v1.0,train,patient00004,study1,view2_lateral.jpg,Female,20,Lateral,NaN,...,0,0,0,0,0,0,0,0,0,1
8,CheXpert-v1.0/train/patient00005/study1/view2_...,CheXpert-v1.0,train,patient00005,study1,view2_lateral.jpg,Male,33,Lateral,NaN,...,0,0,0,0,0,0,0,0,1,0
17,CheXpert-v1.0/train/patient00009/study1/view2_...,CheXpert-v1.0,train,patient00009,study1,view2_lateral.jpg,Male,76,Lateral,NaN,...,0,0,0,0,0,0,0,0,0,0
19,CheXpert-v1.0/train/patient00010/study1/view2_...,CheXpert-v1.0,train,patient00010,study1,view2_lateral.jpg,Female,50,Lateral,NaN,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219186,CheXpert-v1.0/train/patient60703/study1/view1_...,CheXpert-v1.0,train,patient60703,study1,view1_frontal.jpg,Male,88,Frontal,AP,...,1,1,0,1,0,1,0,0,1,0
219188,CheXpert-v1.0/train/patient60704/study1/view1_...,CheXpert-v1.0,train,patient60704,study1,view1_frontal.jpg,Male,67,Frontal,AP,...,0,0,0,0,0,0,0,0,0,1
219192,CheXpert-v1.0/train/patient60706/study1/view1_...,CheXpert-v1.0,train,patient60706,study1,view1_frontal.jpg,Female,99,Frontal,AP,...,1,0,0,0,0,0,0,0,1,0
219194,CheXpert-v1.0/train/patient60707/study1/view1_...,CheXpert-v1.0,train,patient60707,study1,view1_frontal.jpg,Female,38,Frontal,AP,...,0,0,0,0,0,0,0,0,1,0


In [15]:
paired_images=paired_images.sort_index()

In [16]:
paired_images

,Path,Header,Type,Patient,Study,View,Sex,Age,Frontal/Lateral,AP/PA,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
1,CheXpert-v1.0/train/patient00002/study1/view1_...,CheXpert-v1.0,train,patient00002,study1,view1_frontal.jpg,Female,83,Frontal,AP,...,0,1,1,1,0,0,1,1,0,0
2,CheXpert-v1.0/train/patient00002/study1/view2_...,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg,Female,83,Lateral,NaN,...,0,1,1,1,0,0,1,1,0,0
5,CheXpert-v1.0/train/patient00004/study1/view1_...,CheXpert-v1.0,train,patient00004,study1,view1_frontal.jpg,Female,20,Frontal,PA,...,0,0,0,0,0,0,0,0,0,1
6,CheXpert-v1.0/train/patient00004/study1/view2_...,CheXpert-v1.0,train,patient00004,study1,view2_lateral.jpg,Female,20,Lateral,NaN,...,0,0,0,0,0,0,0,0,0,1
7,CheXpert-v1.0/train/patient00005/study1/view1_...,CheXpert-v1.0,train,patient00005,study1,view1_frontal.jpg,Male,33,Frontal,PA,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219193,CheXpert-v1.0/train/patient60706/study1/view2_...,CheXpert-v1.0,train,patient60706,study1,view2_lateral.jpg,Female,99,Lateral,NaN,...,1,0,0,0,0,0,0,0,1,0
219194,CheXpert-v1.0/train/patient60707/study1/view1_...,CheXpert-v1.0,train,patient60707,study1,view1_frontal.jpg,Female,38,Frontal,AP,...,0,0,0,0,0,0,0,0,1,0
219195,CheXpert-v1.0/train/patient60707/study1/view2_...,CheXpert-v1.0,train,patient60707,study1,view2_lateral.jpg,Female,38,Lateral,NaN,...,0,0,0,0,0,0,0,0,1,0
219196,CheXpert-v1.0/train/patient60708/study1/view1_...,CheXpert-v1.0,train,patient60708,study1,view1_frontal.jpg,Female,78,Frontal,AP,...,0,0,0,0,0,0,0,1,0,0


In [17]:
image_path=paired_images.iloc[0,0]
print(image_path)

CheXpert-v1.0/train/patient00002/study1/view1_frontal.jpg


In [18]:
paired_images['1024Path']="DummyPath"

In [19]:
paired_images

,Path,Header,Type,Patient,Study,View,Sex,Age,Frontal/Lateral,AP/PA,...,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,1024Path
1,CheXpert-v1.0/train/patient00002/study1/view1_...,CheXpert-v1.0,train,patient00002,study1,view1_frontal.jpg,Female,83,Frontal,AP,...,1,1,1,0,0,1,1,0,0,DummyPath
2,CheXpert-v1.0/train/patient00002/study1/view2_...,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg,Female,83,Lateral,NaN,...,1,1,1,0,0,1,1,0,0,DummyPath
5,CheXpert-v1.0/train/patient00004/study1/view1_...,CheXpert-v1.0,train,patient00004,study1,view1_frontal.jpg,Female,20,Frontal,PA,...,0,0,0,0,0,0,0,0,1,DummyPath
6,CheXpert-v1.0/train/patient00004/study1/view2_...,CheXpert-v1.0,train,patient00004,study1,view2_lateral.jpg,Female,20,Lateral,NaN,...,0,0,0,0,0,0,0,0,1,DummyPath
7,CheXpert-v1.0/train/patient00005/study1/view1_...,CheXpert-v1.0,train,patient00005,study1,view1_frontal.jpg,Male,33,Frontal,PA,...,0,0,0,0,0,0,0,1,0,DummyPath
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219193,CheXpert-v1.0/train/patient60706/study1/view2_...,CheXpert-v1.0,train,patient60706,study1,view2_lateral.jpg,Female,99,Lateral,NaN,...,0,0,0,0,0,0,0,1,0,DummyPath
219194,CheXpert-v1.0/train/patient60707/study1/view1_...,CheXpert-v1.0,train,patient60707,study1,view1_frontal.jpg,Female,38,Frontal,AP,...,0,0,0,0,0,0,0,1,0,DummyPath
219195,CheXpert-v1.0/train/patient60707/study1/view2_...,CheXpert-v1.0,train,patient60707,study1,view2_lateral.jpg,Female,38,Lateral,NaN,...,0,0,0,0,0,0,0,1,0,DummyPath
219196,CheXpert-v1.0/train/patient60708/study1/view1_...,CheXpert-v1.0,train,patient60708,study1,view1_frontal.jpg,Female,78,Frontal,AP,...,0,0,0,0,0,0,1,0,0,DummyPath


In [25]:
paired_images.iloc[0,24]="Test"

In [28]:
paired_images

,Path,Header,Type,Patient,Study,View,Sex,Age,Frontal/Lateral,AP/PA,...,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,1024Path
1,CheXpert-v1.0/train/patient00002/study1/view1_...,CheXpert-v1.0,train,patient00002,study1,view1_frontal.jpg,Female,83,Frontal,AP,...,1,1,1,0,0,1,1,0,0,Frontal\patient00002_study1_Frontal.png
2,CheXpert-v1.0/train/patient00002/study1/view2_...,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg,Female,83,Lateral,NaN,...,1,1,1,0,0,1,1,0,0,Lateral\patient00002_study1_Lateral.png
5,CheXpert-v1.0/train/patient00004/study1/view1_...,CheXpert-v1.0,train,patient00004,study1,view1_frontal.jpg,Female,20,Frontal,PA,...,0,0,0,0,0,0,0,0,1,Frontal\patient00004_study1_Frontal.png
6,CheXpert-v1.0/train/patient00004/study1/view2_...,CheXpert-v1.0,train,patient00004,study1,view2_lateral.jpg,Female,20,Lateral,NaN,...,0,0,0,0,0,0,0,0,1,Lateral\patient00004_study1_Lateral.png
7,CheXpert-v1.0/train/patient00005/study1/view1_...,CheXpert-v1.0,train,patient00005,study1,view1_frontal.jpg,Male,33,Frontal,PA,...,0,0,0,0,0,0,0,1,0,Frontal\patient00005_study1_Frontal.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219193,CheXpert-v1.0/train/patient60706/study1/view2_...,CheXpert-v1.0,train,patient60706,study1,view2_lateral.jpg,Female,99,Lateral,NaN,...,0,0,0,0,0,0,0,1,0,DummyPath
219194,CheXpert-v1.0/train/patient60707/study1/view1_...,CheXpert-v1.0,train,patient60707,study1,view1_frontal.jpg,Female,38,Frontal,AP,...,0,0,0,0,0,0,0,1,0,DummyPath
219195,CheXpert-v1.0/train/patient60707/study1/view2_...,CheXpert-v1.0,train,patient60707,study1,view2_lateral.jpg,Female,38,Lateral,NaN,...,0,0,0,0,0,0,0,1,0,DummyPath
219196,CheXpert-v1.0/train/patient60708/study1/view1_...,CheXpert-v1.0,train,patient60708,study1,view1_frontal.jpg,Female,78,Frontal,AP,...,0,0,0,0,0,0,1,0,0,DummyPath


In [29]:
res=1024
size_tuple=(res,res)
res_dir="G:\\Big Download\\CheXpert-v1.0\\1024px Images\\"
save_frmt=".png"

for i in range(61266):
    
    #Load image
    org_path=paired_images.iloc[i,0]
    org_filepath="G:\\Big Download\\"+org_path

    #i_image=np.asarray(io.imread(org_filepath))
    i_image=Image.open(org_filepath)

    #Resize image to 1x1 at specified size
    #i_image_res=resize(i_image,(res,res),anti_aliasing=True)
    i_image_res=i_image.resize(size_tuple,resample=1)


    view=paired_images.iloc[i,8]
    file_string=paired_images.iloc[i,3] + '_' + paired_images.iloc[i,4] + '_' + paired_images.iloc[i,8]

    save_path_aft=view + "\\" + file_string + save_frmt
    save_path=res_dir + save_path_aft

    #io.imsave(save_path,i_image_res)
    i_image_res.save(save_path)

    #Save File Path
    paired_images.iloc[i,24]=save_path_aft


In [20]:
res=128
res2=64
size_tuple=(res,res)
size_tuple2=(res2,res2)
res_dir="G:\\Big Download\\CheXpert-v1.0\\128px Images\\"
res_dir2="G:\\Big Download\\CheXpert-v1.0\\64px Images\\"
save_frmt=".png"

for i in range(61266):
    
    #Load image
    org_path=paired_images.iloc[i,0]
    org_filepath="G:\\Big Download\\"+org_path

    #i_image=np.asarray(io.imread(org_filepath))
    i_image=Image.open(org_filepath)

    #Resize image to 1x1 at specified size
    #i_image_res=resize(i_image,(res,res),anti_aliasing=True)
    i_image_res=i_image.resize(size_tuple,resample=1)
    i_image_res2=i_image.resize(size_tuple2,resample=1)



    view=paired_images.iloc[i,8]
    file_string=paired_images.iloc[i,3] + '_' + paired_images.iloc[i,4] + '_' + paired_images.iloc[i,8]

    save_path_aft=view + "\\" + file_string + save_frmt

    save_path=res_dir + save_path_aft
    save_path2=res_dir2 + save_path_aft

    #io.imsave(save_path,i_image_res)
    i_image_res.save(save_path)
    i_image_res2.save(save_path2)

    #Save File Path
    paired_images.iloc[i,24]=save_path_aft


In [31]:
paired_images.to_csv("train_labels_128p.csv")

In [32]:
paired_images['512ath']="DummyPath"
paired_images

,Path,Header,Type,Patient,Study,View,Sex,Age,Frontal/Lateral,AP/PA,...,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,1024Path,512ath
1,CheXpert-v1.0/train/patient00002/study1/view1_...,CheXpert-v1.0,train,patient00002,study1,view1_frontal.jpg,Female,83,Frontal,AP,...,1,1,0,0,1,1,0,0,Frontal\patient00002_study1_Frontal.png,DummyPath
2,CheXpert-v1.0/train/patient00002/study1/view2_...,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg,Female,83,Lateral,NaN,...,1,1,0,0,1,1,0,0,Lateral\patient00002_study1_Lateral.png,DummyPath
5,CheXpert-v1.0/train/patient00004/study1/view1_...,CheXpert-v1.0,train,patient00004,study1,view1_frontal.jpg,Female,20,Frontal,PA,...,0,0,0,0,0,0,0,1,Frontal\patient00004_study1_Frontal.png,DummyPath
6,CheXpert-v1.0/train/patient00004/study1/view2_...,CheXpert-v1.0,train,patient00004,study1,view2_lateral.jpg,Female,20,Lateral,NaN,...,0,0,0,0,0,0,0,1,Lateral\patient00004_study1_Lateral.png,DummyPath
7,CheXpert-v1.0/train/patient00005/study1/view1_...,CheXpert-v1.0,train,patient00005,study1,view1_frontal.jpg,Male,33,Frontal,PA,...,0,0,0,0,0,0,1,0,Frontal\patient00005_study1_Frontal.png,DummyPath
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219193,CheXpert-v1.0/train/patient60706/study1/view2_...,CheXpert-v1.0,train,patient60706,study1,view2_lateral.jpg,Female,99,Lateral,NaN,...,0,0,0,0,0,0,1,0,Lateral\patient60706_study1_Lateral.png,DummyPath
219194,CheXpert-v1.0/train/patient60707/study1/view1_...,CheXpert-v1.0,train,patient60707,study1,view1_frontal.jpg,Female,38,Frontal,AP,...,0,0,0,0,0,0,1,0,Frontal\patient60707_study1_Frontal.png,DummyPath
219195,CheXpert-v1.0/train/patient60707/study1/view2_...,CheXpert-v1.0,train,patient60707,study1,view2_lateral.jpg,Female,38,Lateral,NaN,...,0,0,0,0,0,0,1,0,Lateral\patient60707_study1_Lateral.png,DummyPath
219196,CheXpert-v1.0/train/patient60708/study1/view1_...,CheXpert-v1.0,train,patient60708,study1,view1_frontal.jpg,Female,78,Frontal,AP,...,0,0,0,0,0,1,0,0,Frontal\patient60708_study1_Frontal.png,DummyPath


In [33]:
paired_images['256path']="DummyPath"
paired_images

,Path,Header,Type,Patient,Study,View,Sex,Age,Frontal/Lateral,AP/PA,...,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,1024Path,512ath,256path
1,CheXpert-v1.0/train/patient00002/study1/view1_...,CheXpert-v1.0,train,patient00002,study1,view1_frontal.jpg,Female,83,Frontal,AP,...,1,0,0,1,1,0,0,Frontal\patient00002_study1_Frontal.png,DummyPath,DummyPath
2,CheXpert-v1.0/train/patient00002/study1/view2_...,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg,Female,83,Lateral,NaN,...,1,0,0,1,1,0,0,Lateral\patient00002_study1_Lateral.png,DummyPath,DummyPath
5,CheXpert-v1.0/train/patient00004/study1/view1_...,CheXpert-v1.0,train,patient00004,study1,view1_frontal.jpg,Female,20,Frontal,PA,...,0,0,0,0,0,0,1,Frontal\patient00004_study1_Frontal.png,DummyPath,DummyPath
6,CheXpert-v1.0/train/patient00004/study1/view2_...,CheXpert-v1.0,train,patient00004,study1,view2_lateral.jpg,Female,20,Lateral,NaN,...,0,0,0,0,0,0,1,Lateral\patient00004_study1_Lateral.png,DummyPath,DummyPath
7,CheXpert-v1.0/train/patient00005/study1/view1_...,CheXpert-v1.0,train,patient00005,study1,view1_frontal.jpg,Male,33,Frontal,PA,...,0,0,0,0,0,1,0,Frontal\patient00005_study1_Frontal.png,DummyPath,DummyPath
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219193,CheXpert-v1.0/train/patient60706/study1/view2_...,CheXpert-v1.0,train,patient60706,study1,view2_lateral.jpg,Female,99,Lateral,NaN,...,0,0,0,0,0,1,0,Lateral\patient60706_study1_Lateral.png,DummyPath,DummyPath
219194,CheXpert-v1.0/train/patient60707/study1/view1_...,CheXpert-v1.0,train,patient60707,study1,view1_frontal.jpg,Female,38,Frontal,AP,...,0,0,0,0,0,1,0,Frontal\patient60707_study1_Frontal.png,DummyPath,DummyPath
219195,CheXpert-v1.0/train/patient60707/study1/view2_...,CheXpert-v1.0,train,patient60707,study1,view2_lateral.jpg,Female,38,Lateral,NaN,...,0,0,0,0,0,1,0,Lateral\patient60707_study1_Lateral.png,DummyPath,DummyPath
219196,CheXpert-v1.0/train/patient60708/study1/view1_...,CheXpert-v1.0,train,patient60708,study1,view1_frontal.jpg,Female,78,Frontal,AP,...,0,0,0,0,1,0,0,Frontal\patient60708_study1_Frontal.png,DummyPath,DummyPath


In [35]:
res=512
res2=256
size_tuple=(res,res)
size_tuple2=(res2,res2)
res_dir="G:\\Big Download\\CheXpert-v1.0\\512px Images\\"
res_dir2="G:\\Big Download\\CheXpert-v1.0\\256px Images\\"
save_frmt=".png"

for i in range(61266):
    
    #Load image
    org_path=paired_images.iloc[i,0]
    org_filepath="G:\\Big Download\\"+org_path

    #i_image=np.asarray(io.imread(org_filepath))
    i_image=Image.open(org_filepath)

    #Resize image to 1x1 at specified size
    #i_image_res=resize(i_image,(res,res),anti_aliasing=True)
    i_image_res=i_image.resize(size_tuple,resample=1)
    i_image_res2=i_image.resize(size_tuple2,resample=1)



    view=paired_images.iloc[i,8]
    file_string=paired_images.iloc[i,3] + '_' + paired_images.iloc[i,4] + '_' + paired_images.iloc[i,8]

    save_path_aft=view + "\\" + file_string + save_frmt

    save_path=res_dir + save_path_aft
    save_path2=res_dir2 + save_path_aft

    #io.imsave(save_path,i_image_res)
    i_image_res.save(save_path)
    i_image_res2.save(save_path2)

    #Save File Path
    paired_images.iloc[i,25]=save_path_aft
    paired_images.iloc[i,26]=save_path_aft


In [36]:
paired_images.to_csv("train_labels_512p.csv")

In [ ]:
print(paired_images.iloc[0,3])

In [ ]:
file_string=paired_images.iloc[0,3] + '_' + paired_images.iloc[0,4] + '_' + paired_images.iloc[0,8]
print(file_string)

In [ ]:
image_filepath="G:\\Big Download\\"+image_path

In [ ]:
print(image_filepath)

In [ ]:
test_image=np.asarray(io.imread(image_filepath))
plt.imshow(test_image)

In [ ]:
test_image.shape

In [ ]:
test_image_sizeone=resize(test_image,(1024,1024),anti_aliasing=True)
plt.imshow(test_image_sizeone)

In [ ]:
save_path_one="G:\\Big Download\\CheXpert-v1.0\\1024px Images\Lateral"
save_img_path=save_path_one+"\\test_image.png"

In [ ]:
io.imsave(save_img_path,test_image_sizeone)

In [5]:
csv_path = "256pxImages/train_labels_256p.csv"
myCSV = pd.read_csv(csv_path)
myCSV

,256path,Patient,Study,Frontal/Lateral,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,Frontal\patient00002_study1_Frontal.png,patient00002,study1,Frontal,1,1,1,1,0,1,1,1,0,0,1,1,0,0
1,Lateral\patient00002_study1_Lateral.png,patient00002,study1,Lateral,1,1,1,1,0,1,1,1,0,0,1,1,0,0
2,Frontal\patient00004_study1_Frontal.png,patient00004,study1,Frontal,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Lateral\patient00004_study1_Lateral.png,patient00004,study1,Lateral,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,Frontal\patient00005_study1_Frontal.png,patient00005,study1,Frontal,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61261,Lateral\patient60706_study1_Lateral.png,patient60706,study1,Lateral,1,1,1,0,1,0,0,0,0,0,0,0,1,0
61262,Frontal\patient60707_study1_Frontal.png,patient60707,study1,Frontal,1,1,0,0,0,0,0,0,0,0,0,0,1,0
61263,Lateral\patient60707_study1_Lateral.png,patient60707,study1,Lateral,1,1,0,0,0,0,0,0,0,0,0,0,1,0
61264,Frontal\patient60708_study1_Frontal.png,patient60708,study1,Frontal,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [6]:
for i in range(61266):
    
    #Load image
    org_path=myCSV.iloc[i,0]
    org_filepath="256pxImages/"+org_path
    i_image=Image.open(org_filepath)